In [142]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer, KNNImputer

In [143]:
import pickle

# Abre o arquivo .pkl usando pickle
with open('Titanic.pkl', 'rb') as f:
    data = pickle.load(f)

# Verifica a estrutura dos dados carregados
print("Tipo de dados carregados:", type(data))
print("Conteúdo dos dados:", data)

# Se for uma lista, acesse os elementos por índice
if isinstance(data, list):
    X_treino = data[0]
    X_teste = data[1]
    y_treino = data[2]
    y_teste = data[3]
    feature_names = data[4] if len(data) > 4 else None  # Verifica se feature_names existe
else:
    raise TypeError("O arquivo .pkl não contém uma lista. Verifique a estrutura dos dados.")

# Verifica os dados carregados
print("X_treino:", X_treino)
print("X_teste:", X_teste)
print("y_treino:", y_treino)
print("y_teste:", y_teste)
print("Feature Names:", feature_names)

Tipo de dados carregados: <class 'list'>
Conteúdo dos dados: [     Pclass  Sex   Age  SibSp  Parch      Fare
331       1    1  45.5      0      0   28.5000
733       2    1  23.0      0      0   13.0000
382       3    1  32.0      0      0    7.9250
704       3    1  26.0      1      0    7.8542
813       3    0   6.0      4      2   31.2750
..      ...  ...   ...    ...    ...       ...
106       3    0  21.0      0      0    7.6500
270       1    1  30.0      0      0   31.0000
860       3    1  41.0      2      0   14.1083
435       1    0  14.0      1      2  120.0000
102       1    1  21.0      0      1   77.2875

[712 rows x 6 columns],      Pclass  Sex   Age  SibSp  Parch     Fare
709       3    1  30.0      1      1  15.2458
439       2    1  31.0      0      0  10.5000
840       3    1  20.0      0      0   7.9250
720       2    0   6.0      0      1  33.0000
39        3    0  14.0      1      0  11.2417
..      ...  ...   ...    ...    ...      ...
433       3    1  17.0     

In [144]:
## Método 1: Imputação por Mediana (para Age) e Média (para Fare)
median_imputer = SimpleImputer(strategy='median')  # Age tem outliers
mean_imputer = SimpleImputer(strategy='mean')      # Fare

X_treino_median = X_treino.copy()
X_teste_median = X_teste.copy()

X_treino_median['Age'] = median_imputer.fit_transform(X_treino[['Age']])
X_teste_median['Age'] = median_imputer.transform(X_teste[['Age']])

X_treino_median['Fare'] = mean_imputer.fit_transform(X_treino[['Fare']])
X_teste_median['Fare'] = mean_imputer.transform(X_teste[['Fare']])

# Treinar modelo com imputação por média/mediana
model = RandomForestClassifier(random_state=42)
model.fit(X_treino_median, y_treino)
y_pred_median = model.predict(X_teste_median)

In [145]:
print("=== Imputação por Média/Mediana ===")
print(classification_report(y_teste, y_pred_median))

=== Imputação por Média/Mediana ===
              precision    recall  f1-score   support

           0       0.81      0.86      0.83       105
           1       0.78      0.72      0.75        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



In [146]:
## Método 2: KNNImputer (imputação multivariada)
knn_imputer = KNNImputer(n_neighbors=5)

X_treino_knn = X_treino.copy()
X_teste_knn = X_teste.copy()

# Aplicar KNNImputer a todas as colunas numéricas
X_treino_knn[['Age', 'Fare']] = knn_imputer.fit_transform(X_treino[['Age', 'Fare']])
X_teste_knn[['Age', 'Fare']] = knn_imputer.transform(X_teste[['Age', 'Fare']])

# Treinar modelo com KNNImputer
model = RandomForestClassifier(random_state=42)
model.fit(X_treino_knn, y_treino)
y_pred_knn = model.predict(X_teste_knn)

In [147]:
print("\n=== KNNImputer ===")
print(classification_report(y_teste, y_pred_knn))


=== KNNImputer ===
              precision    recall  f1-score   support

           0       0.81      0.86      0.83       105
           1       0.78      0.72      0.75        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



In [ ]:
# Transforma em DataFrame para facilitar
X_treino_df = pd.DataFrame(X_treino, columns=feature_names)
X_teste_df = pd.DataFrame(X_teste, columns=feature_names)

# Identifica colunas numéricas e categóricas
num_cols = X_treino_df.select_dtypes(include=[np.number]).columns
cat_cols = X_treino_df.select_dtypes(exclude=[np.number]).columns

# Função para testar imputação e gerar o classification_report
def testar_imputacao(nome_metodo, imputer_num):
    print(f"\n=== Método de imputação: {nome_metodo} ===")

    # Cria pré-processador com imputação adequada por tipo
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', imputer_num, num_cols),
            ('cat', Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OneHotEncoder(handle_unknown='ignore'))
            ]), cat_cols)
        ]
    )

    # Cria o pipeline com pre-processamento e o classificador
    modelo = Pipeline(steps=[
        ('preprocessamento', preprocessor),
        ('classificador', DecisionTreeClassifier(random_state=42))
    ])

    modelo.fit(X_treino_df, y_treino)
    y_pred = modelo.predict(X_teste_df)

    acc = accuracy_score(y_teste, y_pred)
    print(f"Acurácia: {acc:.4f}")
    print("Classification Report:")
    print(classification_report(y_teste, y_pred, zero_division=0))

    return acc

# Imputação por média (numérica) e moda (categórica)
acc_media_moda = testar_imputacao("MÉDIA/MODA", SimpleImputer(strategy="mean"))

# Imputação com KNN
acc_knn = testar_imputacao("KNN", KNNImputer(n_neighbors=5))

# Resultado final
melhor = "MÉDIA/MODA" if acc_media_moda > acc_knn else "KNN"
print(f"\n>>> O melhor resultado foi com imputação por {melhor}. <<<")



=== Método de imputação: MÉDIA/MODA ===
Acurácia: 0.7709
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.83      0.81       105
           1       0.74      0.69      0.71        74

    accuracy                           0.77       179
   macro avg       0.77      0.76      0.76       179
weighted avg       0.77      0.77      0.77       179


=== Método de imputação: KNN ===
Acurácia: 0.7709
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.83      0.81       105
           1       0.74      0.69      0.71        74

    accuracy                           0.77       179
   macro avg       0.77      0.76      0.76       179
weighted avg       0.77      0.77      0.77       179


>>> O melhor resultado foi com imputação por KNN. <<<
